In [10]:
pip install opencv-python numpy pytesseract Pillow


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import numpy as np

def correct_orientation(image_path):
    # Load image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Edge detection + HoughLines to find text angle
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)
    
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angles.append(angle)
    
    # Check if text is vertical (angles near 90° or -90°)
    median_angle = np.median(angles)
    if abs(median_angle) > 45:  # Vertical text detected
        if median_angle > 0:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    else:
        rotated_img = img  # Already horizontal
    
    return rotated_img

# Usage
corrected_img = correct_orientation("test.jpg")
cv2.imwrite("horizontal_text.jpg", corrected_img)

True

In [16]:
from PIL import Image
import pytesseract

def rotate_image(image_path, output_path="rotated.jpg"):
    try:
        img = Image.open(image_path)
        
        # Check orientation (simplified for 90° or 270° rotations)
        try:
            osd = pytesseract.image_to_osd(img)
            angle = int(osd.split("Rotate: ")[1].split("\n")[0])
            if angle == 90:
                img = img.rotate(-90, expand=True)
            elif angle == 270:
                img = img.rotate(90, expand=True)
        except:
            # Fallback: Manual rotation (if OSD fails)
            img = img.rotate(90, expand=True)  # Adjust angle as needed
        
        img.save(output_path)
        return output_path
    except Exception as e:
        print(f"Rotation failed: {e}")
        return None

# Usage
rotated_path = rotate_image("test.jpg")